In [1]:
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


#### Setup the web scraping object 

In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

#### Scarpe the web for the neighborhood information

In [49]:
table=soup.find('table')
table_contents=[]

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        # row.span.text is in this format: 'North York(Parkwoods)', we need to extract 'Parkwoods' out of it
        raw_text = row.span.text
        cell['Neighborhood'] = raw_text[raw_text.index('(')+1:raw_text.index(')')].replace(' / ', ', ')
        table_contents.append(cell)

#### Create the dataframe that holds all the information

In [55]:
toronto_neighborhood_df = pd.DataFrame(table_contents)
toronto_neighborhood_df['Borough']=toronto_neighborhood_df['Borough'].replace(
    {'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'}
)
toronto_neighborhood_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


#### Check for not assigned neighborhood

In [43]:
not_assigned_neighborhood = toronto_neighborhood_df[toronto_neighborhood_df['Neighborhood'] == 'Not assigned']
print('There are {} unassigned neighborhood(s)'.format(len(not_assigned_neighborhood)))

There are 0 unassigned neighborhood(s)


In [56]:
print('Shape of the result dataframe: {}'.format(toronto_neighborhood_df.shape))

Shape of the result dataframe: (103, 3)
